In [ ]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain_community.document_loaders import UnstructuredFileLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

# 1) LLM
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1,
)

memory = ConversationBufferMemory(
    llm=llm,
    return_messages=True,
)

# 2) 캐시 저장소
cache_dir = LocalFileStore("./.cache/")

# 3) 텍스트 스플리터
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

# 4) 문서 로드
loader = UnstructuredFileLoader("./files/chapter_three.txt", mode="single")
docs = loader.load()
splits = splitter.split_documents(docs)

# 5) 임베딩 + 캐시
embeddings = OpenAIEmbeddings()
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

# 6) 벡터스토어 & 리트리버
vectorstore = FAISS.from_documents(splits, cached_embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})  

# 7) 프롬프트
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. "
            "If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        ("human", "{question}"),
    ]
)

# 문서 리스트 → 문자열로 변환
# def format_docs(docs):
#     return "\n\n".join(d.page_content for d in docs)

# 8) 체인 (문서 포맷팅 + 출력 파서 추가)

chain = (
    {
    # | RunnableLambda(format_docs)
        "context": retriever ,  # ← 핵심 수정
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()  
)


C:\Users\kwon\AppData\Local\Temp\ipykernel_26188\3851760724.py:29: LangChainDeprecationWarning: The class `UnstructuredFileLoader` was deprecated in LangChain 0.2.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-unstructured package and should be used instead. To use it run `pip install -U :class:`~langchain-unstructured` and import as `from :class:`~langchain_unstructured import UnstructuredLoader``.
  loader = UnstructuredFileLoader("./files/chapter_three.txt", mode="single")
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
c:\Users\kwon\AppData\Local\pypoetry\Cache\virtualenvs\langchain-kr--P7Goz9u-py3.11\Lib\site-packages\langchain\embeddings\cache.py:58: UserWarning: Using default key encoder: SHA-1 is *not* collision-resistant. While acceptable for most cache scenarios, a motivated attacker can craft two different payloads that map to the same cache key. If that risk ma

In [3]:

result = chain.invoke("Is Aaronson guilty?")
print(result)  # ChatOpenAI의 반환은 Message이므로 .contentfrom langchain.memory import ConversationSummaryBufferMemory, ConversationBufferMemory


According to the context, Winston believes that Aaronson, along with Jones and Rutherford, is guilty of the crimes they are charged with, even though he has never seen the photograph that disproved their guilt and acknowledges that it had never existed.


In [ ]:
result = chain.invoke("Is Aaronson guilty?")
print(result)

In [4]:
result = chain.invoke("What message did Aaronson write on the table?")
print(result)

I don't know.


In [5]:
result = chain.invoke("Who is Julia")
print(result)

Julia is a character who is significant to the protagonist, Winston. She is someone he loves and has a deep emotional connection with, as indicated by Winston's overwhelming feelings for her during moments of reverie and distress.
